In [ ]:
#from TRAIL24.compression.pruning import *

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from torchmetrics import MeanSquaredError, SymmetricMeanAbsolutePercentageError
from torch.utils.data import DataLoader, Dataset
import time
#from memory_profiler import memory_usage
import resource
import pickle
import gc
import warnings
from tqdm.notebook import tqdm

from torchmetrics import MeanSquaredError, SymmetricMeanAbsolutePercentageError


import warnings
warnings.filterwarnings('ignore')

In [ ]:

with open('../data/df_cluster_0.pkl', 'rb') as f:
    df = pickle.load(f)


# In[5]:

print('OKKKK')

num_rows = len(df)

# Calculate the halfway point
halfway_point = num_rows // 4

# Select the first half of the rows
df = df.iloc[:halfway_point]




import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# Function to optimize data types
def optimize_data_types(df):
    # Optimize numeric columns
    for col in df.select_dtypes(include=['int']).columns:
        df[col] = df[col].astype('int32')
    
    for col in df.select_dtypes(include=['float']).columns:
        df[col] = df[col].astype('float32')

    # Optimize object columns
    for col in df.select_dtypes(include=['object']).columns:
        num_unique_values = len(df[col].unique())
        num_total_values = len(df[col])
        if num_unique_values / num_total_values < 0.5:
            df[col] = df[col].astype('category')
    
    return df

# Load the data and remove the first and last columns
#df = pd.read_csv('df_labeled0.csv')
df = df.iloc[:, 1:-1]

# Optimize data types
df = optimize_data_types(df)

# Define the window size for lagged inputs
window_size = 40  # Replace N with your desired window size

# Function to process each chunk
def process_chunk(chunk, window_size):
    data = []
    for i, row in tqdm(chunk.iterrows()):
        values = row.values
        for t in range(window_size, len(values)):
            lagged_inputs = values[t-window_size:t]
            target_value = values[t]

            # Calculate realistic datetime values using the index
            base_date = pd.Timestamp('2009-07-14 00:00:00')
            timestamp = base_date + pd.Timedelta(hours=t)
            year = timestamp.year
            month = timestamp.month
            day = timestamp.day
            hour = timestamp.hour

            data.append(list(lagged_inputs) + [year, month, day, hour, target_value])
    return data

# Create an empty DataFrame to store results
columns = [f'lag_{i}' for i in range(1, window_size+1)] + ['year', 'month', 'day', 'hour', 'target']
df_transformed = pd.DataFrame(columns=columns)

# Process the data in chunks
chunk_size = 100  # Adjust based on available memory
for start_row in tqdm(range(0, df.shape[0], chunk_size)):
    chunk = df.iloc[start_row:start_row + chunk_size]
    data = process_chunk(chunk, window_size)
    df_chunk_transformed = pd.DataFrame(data, columns=columns)
    df_transformed = pd.concat([df_transformed, df_chunk_transformed], ignore_index=True)

# Make sure year, month, day, and hour are integers
df_transformed['year'] = df_transformed['year'].astype(int)
df_transformed['month'] = df_transformed['month'].astype(int)
df_transformed['day'] = df_transformed['day'].astype(int)
df_transformed['hour'] = df_transformed['hour'].astype(int)

# Create datetime column
try:
    df_transformed['date_time'] = pd.to_datetime(df_transformed[['year', 'month', 'day', 'hour']])
    print("Date conversion successful.")
except Exception as e:
    print("Error during date conversion:", e)

# Apply OrdinalEncoder to month, day, hour columns
ordinal_encoder = OrdinalEncoder()
df_transformed[['month', 'day', 'hour']] = ordinal_encoder.fit_transform(df_transformed[['month', 'day', 'hour']])

print("Transformation complete. The new dataset is saved as 'transformed_dataset.csv'.")




print(df_transformed[['year', 'month', 'day', 'hour']].dtypes)


# In[9]:


print(df_transformed['month'].unique())  # Should be within 1-12
print(df_transformed['day'].unique())    # Should be within 1-31
print(df_transformed['hour'].unique())   # Should be within 0-23


# In[10]:


# Convert the date columns to datetime
df_transformed['year1'] = df_transformed['year']
df_transformed['month1'] = df_transformed['month']
df_transformed['day1'] = df_transformed['day']
df_transformed['hour1'] = df_transformed['hour']
df_transformed["target1"] = df_transformed["target"]

# Define the date ranges for training and testing
train_start_date = '2009-07-14'
train_end_date = '2010-12-15'
test_start_date = '2010-12-15'
test_end_date = '2011-01-01'

# Convert the date columns to datetime
df_transformed['year'] = df_transformed['year'].astype(int)
df_transformed['month'] = df_transformed['month'].astype(int)+1
df_transformed['day'] = df_transformed['day'].astype(int)+1
df_transformed['hour'] = df_transformed['hour'].astype(int)+1

# Create a datetime column
df_transformed['date_time'] = pd.to_datetime(df_transformed[['year', 'month', 'day', 'hour']])
df_sample = df_transformed.loc[:12825]

# Set the datetime column as the index
df_transformed.set_index('date_time', inplace=True)
df_sample.set_index('date_time', inplace=True)

# Drop the irrelevant columns
df_transformed.drop(columns=['year', 'month', 'day', 'hour','target'], inplace=True)
df_sample.drop(columns=['year', 'month', 'day', 'hour','target'], inplace=True)

df_transformed = df_transformed.sort_index()
df_sample = df_sample.sort_index()

# Split the data into training and testing sets
train = df_transformed.loc[train_start_date:train_end_date]

test = df_transformed.loc[test_start_date:test_end_date]

scaler = StandardScaler()

# Fit the scaler on the first 40 columns of the training data
scaler.fit(train.iloc[:, :40])

# Transform the first 40 columns of both training and validation data
train.iloc[:, :40] = scaler.transform(train.iloc[:, :40])
test.iloc[:, :40] = scaler.transform(test.iloc[:, :40])

# Create sample data for the first building
sample = df_sample.loc[test_start_date:test_end_date]

sample.iloc[:, :40] = scaler.transform(sample.iloc[:, :40])

del df_transformed, df
gc.collect()

OKKKK


  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Date conversion successful.
Transformation complete. The new dataset is saved as 'transformed_dataset.csv'.
year       int64
month    float64
day      float64
hour     float64
dtype: object
[ 6.  7.  8.  9. 10. 11.  0.  1.  2.  3.  4.  5.]
[14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30.  0.
  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
[16. 17. 18. 19. 20. 21. 22. 23.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9.
 10. 11. 12. 13. 14. 15.]


SettingWithCopyError: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [ ]:
from sklearn.preprocessing import StandardScaler
# Instantiate a StandardScaler
scaler = StandardScaler()

# Standardize only the first 40 columns
sample.iloc[:, :40] = scaler.fit_transform(sample.iloc[:, :40])

# Display the updated DataFrame
print(df)

     2009-07-14 00:00:00  2009-07-14 01:00:00  2009-07-14 02:00:00  \
1                  0.692                0.761                0.725   
2                  1.310                2.360                1.693   
3                  0.177                0.324                0.317   
4                  0.048                0.183                0.101   
5                  0.614                1.142                1.139   
..                   ...                  ...                  ...   
908                0.128                0.297                0.284   
909                0.229                0.402                0.348   
912                0.626                0.691                0.375   
913                0.179                0.183                0.178   
915                0.448                0.372                0.292   

     2009-07-14 03:00:00  2009-07-14 04:00:00  2009-07-14 05:00:00  \
1                  0.546                0.729                0.755   
2                  

In [ ]:
df_transformed.iloc[:, :40].max()

lag_1     24.517
lag_2     24.517
lag_3     24.517
lag_4     24.517
lag_5     24.517
lag_6     24.517
lag_7     24.517
lag_8     24.517
lag_9     24.517
lag_10    24.517
lag_11    24.517
lag_12    24.517
lag_13    24.517
lag_14    24.517
lag_15    24.517
lag_16    24.517
lag_17    24.517
lag_18    24.517
lag_19    24.517
lag_20    24.517
lag_21    24.517
lag_22    24.517
lag_23    24.517
lag_24    24.517
lag_25    24.517
lag_26    24.517
lag_27    24.517
lag_28    24.517
lag_29    24.517
lag_30    24.517
lag_31    24.517
lag_32    24.517
lag_33    24.517
lag_34    24.517
lag_35    24.517
lag_36    24.517
lag_37    24.517
lag_38    24.517
lag_39    24.517
lag_40    24.517
dtype: float32

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,...,lag_36,lag_37,lag_38,lag_39,lag_40,year1,month1,day1,hour1,target1
date_time,,,,,,,,,,,,,,,,,,,,,
2010-12-15 00:00:00,0.057708,0.070939,0.079591,0.056690,0.192157,0.119895,0.046106,0.039592,0.043052,0.093331,...,0.350626,0.184117,0.315105,0.208035,0.301671,2010,11.0,13.0,23.0,3.437
2010-12-15 01:00:00,0.067060,0.075238,0.053590,0.181650,0.113339,0.043584,0.037427,0.040698,0.088227,0.244476,...,0.174049,0.297874,0.196659,0.285174,0.330683,2010,11.0,14.0,0.0,1.874
2010-12-15 02:00:00,0.074206,0.052855,0.179158,0.111784,0.042986,0.036913,0.040140,0.087017,0.241123,0.234385,...,0.293788,0.193961,0.281262,0.326147,0.177829,2010,11.0,14.0,1.0,1.832
2010-12-15 03:00:00,0.052214,0.176984,0.110427,0.042465,0.036465,0.039653,0.085961,0.238197,0.231541,0.327908,...,0.191608,0.277850,0.322189,0.175671,0.171734,2010,11.0,14.0,2.0,0.774
2010-12-15 04:00:00,0.176760,0.110288,0.042411,0.036419,0.039602,0.085852,0.237895,0.231248,0.327492,0.157567,...,0.277498,0.321781,0.175449,0.171517,0.072464,2010,11.0,14.0,3.0,0.723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 20:00:00,0.064250,0.046326,0.036749,0.037895,0.042806,0.088559,0.088886,0.126127,0.183747,0.178836,...,0.066215,0.155592,0.224589,0.328453,0.246115,2010,11.0,30.0,19.0,2.166
2010-12-31 21:00:00,0.045706,0.036258,0.037388,0.042234,0.087374,0.087697,0.124440,0.181289,0.176444,0.216901,...,0.153510,0.221585,0.324060,0.242823,0.174910,2010,11.0,30.0,20.0,1.617
2010-12-31 22:00:00,0.035990,0.037112,0.041921,0.086728,0.087048,0.123519,0.179948,0.175139,0.215297,0.088972,...,0.219946,0.321662,0.241026,0.173616,0.129611,2010,11.0,30.0,21.0,1.636


In [ ]:

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx].astype("float32")
        features = torch.FloatTensor(row[:-5].values)  # All columns except the last 5
        year = torch.FloatTensor([row[-5]]).to(torch.int)
        month = torch.FloatTensor([row[-4]]).to(torch.int)  # Month is expected to be 0-11
        day = torch.FloatTensor([row[-3]]).to(torch.int)    # Day is expected to be 0-30
        hour = torch.FloatTensor([row[-2]]).to(torch.int)      # Hour is already 0-indexed (0-23)
        label = torch.FloatTensor([row[-1]])  # The last column
        return features, year, month, day, hour, label

    def __len__(self):
        return len(self.dataframe)

def create_data_loaders(data, chunk_size, batch_size):
    chunks = np.array_split(data, len(data) // chunk_size)
    data_loaders = []
    for chunk in chunks:
        dataset = CustomDataset(chunk)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        data_loaders.append(data_loader)
    return data_loaders


# In[13]:


def train_function(net, criterion, optimizer, data_loaders, n_epochs=5, device=torch.device("cuda")):
    from torch.optim.lr_scheduler import ReduceLROnPlateau
    scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, threshold=0.1, patience=3, factor=0.5)
    for epoch in tqdm(range(n_epochs)):
        epoch_loss = 0
        counter = 0
        for data_loader in data_loaders:
            for features, year, month, day, hour, labels in tqdm(data_loader):
                features = features.to(device)
                year = year.to(device)
                month = month.to(device)
                day = day.to(device)
                hour = hour.to(device)
                labels = labels.to(device)

                outputs = net(features, month, day, hour, year)
                loss = criterion(outputs, labels)
                epoch_loss += loss
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        print(f"epoch {epoch}")
        scheduler.step(epoch_loss / len(data_loaders))
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(data_loaders)}")
        with open(f'modelcheckpoint{epoch}.pickle', 'wb') as handle:
            pickle.dump([net, optimizer], handle, protocol=pickle.HIGHEST_PROTOCOL)
        collected = gc.collect()
    return net

def test_function(net, data_loaders, scaler, label_scaler, device=torch.device("cuda"), return_data=False):
    mse = MeanSquaredError().to(device)
    smape = SymmetricMeanAbsolutePercentageError().to(device)
    net.eval()
    list_outputs = []
    list_targets = []
    with torch.no_grad():  # to not reserve a memory space for gradients
        for data_loader in data_loaders:
            for features, year, month, day, hour, labels in tqdm(data_loader):
                features = features.to(device)
                year = year.to(device)
                month = month.to(device)
                day = day.to(device)
                hour = hour.to(device)
                labels = labels.to(device)

                outputs = net(features, month, day, hour, year)
                outputs = outputs.squeeze()
                labels = labels.squeeze()

                mse(outputs, labels)
                smape(outputs, labels)
                list_targets.append(labels.detach().unsqueeze(0))  # Ensure tensors are at least one-dimensional
                list_outputs.append(outputs.detach().unsqueeze(0))  # Ensure tensors are at least one-dimensional
    test_mse = mse.compute()
    test_smape = smape.compute()
    print(f"Test MSE: {test_mse} , SMAPE {test_smape}")
    if return_data:
        return torch.cat(list_outputs), torch.cat(list_targets), test_mse, test_smape


In [ ]:

# Define a function to create the model for memory profiling
def create_model():
    model = CombinedModel(nbits_params, embedding_dim, final_hidden)
    return model

def get_memory_usage():
    return resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024

class TrendBlock(nn.Module):
    def __init__(self, input_size, output_size, num_hidden, num_layers):
        super(TrendBlock, self).__init__()
        self.fc = nn.ModuleList([nn.Linear(input_size, num_hidden)] +
                                [nn.Linear(num_hidden, num_hidden) for _ in range(num_layers - 1)])
        self.backcast_fc = nn.Linear(num_hidden, input_size)
        self.forecast_fc = nn.Linear(num_hidden, output_size)

    def forward(self, x):
        for layer in self.fc:
            x = torch.relu(layer(x))
        backcast = self.backcast_fc(x)
        forecast = self.forecast_fc(x)
        return backcast, forecast

class SeasonalityBlock(nn.Module):
    def __init__(self, input_size, output_size, num_hidden, num_layers):
        super(SeasonalityBlock, self).__init__()
        self.fc = nn.ModuleList([nn.Linear(input_size, num_hidden)] +
                                [nn.Linear(num_hidden, num_hidden) for _ in range(num_layers - 1)])
        self.backcast_fc = nn.Linear(num_hidden, input_size)
        self.forecast_fc = nn.Linear(num_hidden, output_size)

    def forward(self, x):
        for layer in self.fc:
            x = torch.relu(layer(x))
        backcast = self.backcast_fc(x)
        forecast = self.forecast_fc(x)
        return backcast, forecast

class NBEATS(nn.Module):
    def __init__(self, input_size, output_size, num_hidden, num_layers, num_blocks):
        super(NBEATS, self).__init__()
        self.trend_blocks = nn.ModuleList([TrendBlock(input_size, output_size, num_hidden, num_layers) for _ in range(num_blocks // 2)])
        self.seasonality_blocks = nn.ModuleList([SeasonalityBlock(input_size, output_size, num_hidden, num_layers) for _ in range(num_blocks // 2)])

    def forward(self, x):
        backcast, forecast = x, 0
        for block in self.trend_blocks:
            backcast, block_forecast = block(x)
            x = x - backcast
            forecast = forecast + block_forecast
        for block in self.seasonality_blocks:
            backcast, block_forecast = block(x)
            x = x - backcast
            forecast = forecast + block_forecast
        return forecast

class EmbeddingNetwork(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(EmbeddingNetwork, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)

    def forward(self, x):
        return self.embedding(x)

class YearNetwork(nn.Module):
    def __init__(self, input_dim, num_hidden):
        super(YearNetwork, self).__init__()
        self.hidden = nn.Linear(input_dim, 25)
        self.output = nn.Linear(25, num_hidden)

    def forward(self, x):
        x = torch.relu(self.hidden(x))
        return self.output(x)

class CombinedModel(nn.Module):
    def __init__(self, nbits_params, embedding_dim, final_hidden):
        super(CombinedModel, self).__init__()
        self.nbeats = NBEATS(**nbits_params)
        self.month_net = EmbeddingNetwork(12, embedding_dim)  # Months from 1 to 12
        self.day_net = EmbeddingNetwork(31, embedding_dim)    # Days from 1 to 31
        self.hour_net = EmbeddingNetwork(24, embedding_dim)   # Hours from 0 to 23
        self.year_net = YearNetwork(1, 10)
        self.final_layer = nn.Sequential(
            #nn.Linear(embedding_dim * 3 + nbits_params['output_size'] + 10, final_hidden),
            nn.Linear(embedding_dim * 3 + nbits_params['output_size'], final_hidden),
            nn.ReLU(),
            nn.Linear(final_hidden, 1)
        )

    def forward(self, x, month, day, hour, year):
        ts_output = self.nbeats(x)
        #print('TS', ts_output)
        month_output = self.month_net(month.long()).squeeze(1)
        #print('Month', month_output)
        day_output = self.day_net(day.long()).squeeze(1)
        #print('Day', day_output)
        hour_output = self.hour_net(hour.long()).squeeze(1)
        #print('Hour', hour_output)
        
        #print('Year', year_output)
        #combined_output = torch.cat((ts_output, month_output, day_output, hour_output, year_output), dim=1)
        combined_output = torch.cat((ts_output, month_output, day_output, hour_output), dim=1)
        final_output = self.final_layer(combined_output)
        return final_output



In [ ]:
input_size = 40  # Length of input time series
output_size = 1  # Length of output time series (forecast)
num_blocks = 12
num_hidden = 512
num_layers = 8
embedding_dim = 10
final_hidden = 256

nbits_params = {
    'input_size': input_size,
    'output_size': output_size,
    'num_blocks': num_blocks,
    'num_hidden': num_hidden,
    'num_layers': num_layers
}

In [ ]:
net = create_model()

In [ ]:
trained_model = torch.load('trained_model.pth')

In [ ]:
trained_model

CombinedModel(
  (nbeats): NBEATS(
    (trend_blocks): ModuleList(
      (0-5): 6 x TrendBlock(
        (fc): ModuleList(
          (0): Linear(in_features=40, out_features=512, bias=True)
          (1-7): 7 x Linear(in_features=512, out_features=512, bias=True)
        )
        (backcast_fc): Linear(in_features=512, out_features=40, bias=True)
        (forecast_fc): Linear(in_features=512, out_features=1, bias=True)
      )
    )
    (seasonality_blocks): ModuleList(
      (0-5): 6 x SeasonalityBlock(
        (fc): ModuleList(
          (0): Linear(in_features=40, out_features=512, bias=True)
          (1-7): 7 x Linear(in_features=512, out_features=512, bias=True)
        )
        (backcast_fc): Linear(in_features=512, out_features=40, bias=True)
        (forecast_fc): Linear(in_features=512, out_features=1, bias=True)
      )
    )
  )
  (month_net): EmbeddingNetwork(
    (embedding): Embedding(12, 10)
  )
  (day_net): EmbeddingNetwork(
    (embedding): Embedding(31, 10)
  )
  (ho

In [ ]:

lr = 0.0005
n_epochs = 10
window_size = 40
chunk_size = 122880
batch_size = 1024*12*5


# In[15]:


# Assuming `train`, `test`, `sample` are pre-loaded DataFrames
train_loaders = create_data_loaders(train, chunk_size, batch_size)
test_loaders = create_data_loaders(test, chunk_size, batch_size)
sample_loaders = create_data_loaders(sample, 100, 1)

device = "cuda"
net.to(device)

CombinedModel(
  (nbeats): NBEATS(
    (trend_blocks): ModuleList(
      (0-5): 6 x TrendBlock(
        (fc): ModuleList(
          (0): Linear(in_features=40, out_features=512, bias=True)
          (1-7): 7 x Linear(in_features=512, out_features=512, bias=True)
        )
        (backcast_fc): Linear(in_features=512, out_features=40, bias=True)
        (forecast_fc): Linear(in_features=512, out_features=1, bias=True)
      )
    )
    (seasonality_blocks): ModuleList(
      (0-5): 6 x SeasonalityBlock(
        (fc): ModuleList(
          (0): Linear(in_features=40, out_features=512, bias=True)
          (1-7): 7 x Linear(in_features=512, out_features=512, bias=True)
        )
        (backcast_fc): Linear(in_features=512, out_features=40, bias=True)
        (forecast_fc): Linear(in_features=512, out_features=1, bias=True)
      )
    )
  )
  (month_net): EmbeddingNetwork(
    (embedding): Embedding(12, 10)
  )
  (day_net): EmbeddingNetwork(
    (embedding): Embedding(31, 10)
  )
  (ho

In [ ]:
*_, test_mse, test_smape = test_function(trained_model, test_loaders, None, None, torch.device(device), return_data=True)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Test MSE: 1.037216067314148 , SMAPE 0.4622192978858948


In [ ]:
test_mse

tensor(1.0372, device='cuda:0')

In [ ]:
test_smape

tensor(0.4622, device='cuda:0')

In [ ]:
import torch_pruning as tp
from torch_pruning.pruner import function
from fasterai.core.all import *

import onnx
import onnxruntime as ort
from onnxruntime import quantization

import numpy as np
import pickle
from itertools import cycle
from fastcore.basics import store_attr, listify, true
from fasterbench.benchmark import *

def get_ignored_layers(model):
    ignored_layers = []

    # Check and process trend blocks if they exist
    if hasattr(model.nbeats, 'trend_blocks'):
        for block in model.nbeats.trend_blocks:
            if hasattr(block, 'backcast_fc'):
                ignored_layers.append(block.backcast_fc)
            if hasattr(block, 'forecast_fc'):
                ignored_layers.append(block.forecast_fc)

    # Check and process seasonality blocks if they exist
    if hasattr(model.nbeats, 'seasonality_blocks'):
        for block in model.nbeats.seasonality_blocks:
            if hasattr(block, 'backcast_fc'):
                ignored_layers.append(block.backcast_fc)
            if hasattr(block, 'forecast_fc'):
                ignored_layers.append(block.forecast_fc)

    return ignored_layers

def adjust_layer_features(layer, pruning_ratio):
    if hasattr(layer, 'in_features') and hasattr(layer, 'weight'):
        in_features = layer.in_features
        layer.in_features = int(in_features * (1-pruning_ratio))
        local_scores = large_final(layer, 'column')
        threshold = torch.quantile(local_scores.view(-1), pruning_ratio)
        mask = local_scores.ge(threshold).to(dtype=local_scores.dtype)
        ixs = torch.nonzero(mask[0] == 1, as_tuple=True)[0]
        layer.weight.data = layer.weight[:, ixs]

def prune_model(model, pruning_ratio, dummy_input):
    imp = tp.importance.GroupNormImportance(p=2)
    ignored_layers = get_ignored_layers(model)

    pruner = tp.pruner.MetaPruner(
        model.nbeats,
        dummy_input,
        importance=imp,
        pruning_ratio=pruning_ratio, 
        ignored_layers=ignored_layers
    )
    pruner.step()
    
    if hasattr(model.nbeats, 'trend_blocks'):
        for block in model.nbeats.trend_blocks:
            for layer in [block.backcast_fc, block.forecast_fc]:
                if layer is not None and layer.in_features != int(num_hidden*(1-pruning_ratio)):
                    adjust_layer_features(layer, pruning_ratio)

    if hasattr(model.nbeats, 'seasonality_blocks'):
        for block in model.nbeats.seasonality_blocks:
            for layer in [block.backcast_fc, block.forecast_fc]:
                if layer is not None and layer.in_features != int(num_hidden*(1-pruning_ratio)):
                    adjust_layer_features(layer, pruning_ratio)

In [ ]:
num_parameters = get_num_parameters(trained_model)
disk_size = get_model_size(trained_model)
print(f"Model Size: {disk_size / 1e6:.2f} MB (disk), {num_parameters} parameters")

NameError: name 'trained_model' is not defined

In [ ]:
batch_size = 5
num_features = 40

features = torch.randn(batch_size, num_features)

In [ ]:
trained_model = torch.load('trained_model.pth')

In [ ]:
prune_model(trained_model, 0.15, features.to('cuda'))

In [ ]:
num_parameters = get_num_parameters(trained_model)
disk_size = get_model_size(trained_model)
print(f"Model Size: {disk_size / 1e6:.2f} MB (disk), {num_parameters} parameters")

Model Size: 65.56 MB (disk), 16369401 parameters


In [ ]:
*_, test_mse, test_smape = test_function(trained_model, test_loaders, None, None, torch.device(device), return_data=True)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Test MSE: 1.1977158784866333 , SMAPE 0.4805765748023987


In [ ]:
test_mse, test_smape

(tensor(1.9425, device='cuda:0'), tensor(0.7663, device='cuda:0'))

In [ ]:
import torch
import torch.nn as nn
from onnxruntime.quantization import quantize_dynamic, QuantType

In [ ]:
def script_model(model, dummy_input, path='scripted_model.pt'):
    scripted_model = torch.jit.trace(model, dummy_input)
    scripted_model.save(path)
    return scripted_model

In [ ]:
def quantize_onnx(model, dummy_input, onnx_path="model.onnx", quant_onnx_path="model_quantized.onnx"):
    torch.onnx.export(
    model,              
    dummy_input,        
    onnx_path, 
    input_names=["features", "year", "month", "day", "hour"],   
    output_names=["output"], 
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}, 
    opset_version=11
    )

    quantize_dynamic(
        onnx_path,
        quant_onnx_path,
        weight_type=QuantType.QUInt8
    )

In [ ]:
batch_size = 5
num_features = 40

features = torch.randn(batch_size, num_features)
month = torch.randint(0, 12, (batch_size, 1))      # Random months between 1 and 12
day = torch.randint(0, 31, (batch_size, 1))        # Random days between 1 and 31
hour = torch.randint(0, 24, (batch_size, 1))       # Random hours between 0 and 23
year = torch.randint(2019, 2021, (batch_size, 1)) 

example_input = features, year, month, day, hour

In [ ]:
trained_model

CombinedModel(
  (nbeats): NBEATS(
    (trend_blocks): ModuleList(
      (0-5): 6 x TrendBlock(
        (fc): ModuleList(
          (0): Linear(in_features=40, out_features=435, bias=True)
          (1-7): 7 x Linear(in_features=435, out_features=435, bias=True)
        )
        (backcast_fc): Linear(in_features=435, out_features=40, bias=True)
        (forecast_fc): Linear(in_features=435, out_features=1, bias=True)
      )
    )
    (seasonality_blocks): ModuleList(
      (0-5): 6 x SeasonalityBlock(
        (fc): ModuleList(
          (0): Linear(in_features=40, out_features=435, bias=True)
          (1-7): 7 x Linear(in_features=435, out_features=435, bias=True)
        )
        (backcast_fc): Linear(in_features=435, out_features=40, bias=True)
        (forecast_fc): Linear(in_features=435, out_features=1, bias=True)
      )
    )
  )
  (month_net): EmbeddingNetwork(
    (embedding): Embedding(12, 10)
  )
  (day_net): EmbeddingNetwork(
    (embedding): Embedding(31, 10)
  )
  (ho

In [ ]:
trained_model.to('cpu')(*example_input)

IndexError: index out of range in self

In [ ]:
scripted_model = script_model(trained_model.to('cpu'), example_input)
quantize_onnx(net, example_input)

IndexError: index out of range in self

In [ ]:
trained_model

CombinedModel(
  (nbeats): NBEATS(
    (trend_blocks): ModuleList(
      (0-5): 6 x TrendBlock(
        (fc): ModuleList(
          (0): Linear(in_features=40, out_features=435, bias=True)
          (1-7): 7 x Linear(in_features=435, out_features=435, bias=True)
        )
        (backcast_fc): Linear(in_features=435, out_features=40, bias=True)
        (forecast_fc): Linear(in_features=435, out_features=1, bias=True)
      )
    )
    (seasonality_blocks): ModuleList(
      (0-5): 6 x SeasonalityBlock(
        (fc): ModuleList(
          (0): Linear(in_features=40, out_features=435, bias=True)
          (1-7): 7 x Linear(in_features=435, out_features=435, bias=True)
        )
        (backcast_fc): Linear(in_features=435, out_features=40, bias=True)
        (forecast_fc): Linear(in_features=435, out_features=1, bias=True)
      )
    )
  )
  (month_net): EmbeddingNetwork(
    (embedding): Embedding(12, 10)
  )
  (day_net): EmbeddingNetwork(
    (embedding): Embedding(31, 10)
  )
  (ho